# Testing for Uniformity of Randomized Search Results

## A Simple Application of a Web Scraper

by

Benjamin Wallace (May 2019)

[OTTIAQ](https://ottiaq.org/en/) is an organization for translators in the Canadian province of Québec. Their website includes a [search engine](https://ottiaq.org/en/services-for-the-public-and-businesses/find-a-professional/) for finding a translator according to criteria such as source language, target language, region, etc. In order to reduce bias towards translators whose last names come have lexicographic precedence, the results of a search query are randomly permuted before being served to the client.

We wish to test whether the returned results are indeed unbiased. To do this, we write a web scraper to repeatedly perform a search and aggregate the ranks of the returned results into an sqlite3 database. The rank distribution can then be tested for uniformity using standard statistical methods (specifically, Pearson's $\chi^2$ test).

## Preliminary note on the ethics of web scraping

While under ordinary circumstances there's nothing illegal about scraping the web, it's best to strive to follow an unwritten code of conduct when using them. This is to avoid overloading the server being scraped, or annoying/arousing suspicion in said server's administrator.

A good place to start is with the site's [robots.txt](https://ottiaq.org/robots.txt). As of May 2019, this file contains no restrictions against scraping the site's search page.

![robots](robots.txt.png)

One could also use the [urllib.robotparser](https://docs.python.org/3/library/urllib.robotparser.html) module to automate this process, but in this case it's not necessary.

## Forming an HTTP request

HTTP requests in Python are easily formed using the third-party [requests](https://2.python-requests.org/en/master/) library (we could also use [urllib.requests](https://docs.python.org/3/library/urllib.request.html), which is part of the Python standard library). We will also make use of the [re](https://docs.python.org/3/library/re.html) module for regular expressions to search through the resulting HTML response (for more involved HTML parsing applications, [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) is a standard choice).

In [1]:
import requests
import re

For the purposes of this demonstration, we will consider the results of a search query with the following parameters: we set the source language to French, the target language to English, and leave all other fields unchanged.

In order to determine how these parameters should be encoded and what kind of HTTP request to make, we need to inspect the search form element. This is easily accomplished using any modern browser's developer tools (though a modern browser isn't strictly necessary). For instance, using Google Chrome's "Inspect Element" tool, we find that the variable carrying the source language information is called `langue_depart_off` (see image below). Similarly, the target variable is `langue_arrivee_off` and the desired values in our case are `FR` and `EN`, respectively. Lastly, we find that the form acts by POSTing data to `https://ottiaq.org/en/directory/results/`.

![Inspect element](inspect.png)

Let's save the information we've gathered into the following variables. Later, we will create a table in an sqlite3 database corresponding to the parameters we've chosen. We save the name of our table here. The procedure outlined below can then easily be adapted to other search queries by modifying the `data` dictionary and the table name.

In [2]:
url = 'https://ottiaq.org/en/directory/results/'
data = {'langue_depart_off': 'FR', 'langue_arrivee_off': 'EN'}
table = 'fren'

By going ahead and performing a search with these parameters, we find that each returned result includes a convenient id specifier prefaced by the phrase "Membership number : ". We will capture this id with the following regular expression and use it as the primary key in our database.

In [3]:
pattern = re.compile("(?<=Membership number : )\d+")

Since we'll need to perform the HTTP request and then extract the id using the regex pattern above many times, we'll package this sequence of operations into the following function.

In [4]:
def scrape(url, data):
    response = requests.post(url, data)
    html = response.text
    matches = pattern.findall(html)
    
    return list(map(int, matches))

## Setting up the database

We'll store the results obtained from `collect` into an sqlite3 database, which we can interface with through the [sqlite3](https://docs.python.org/2/library/sqlite3.html) module.

In [5]:
import sqlite3

The pertinent information we would like to save is list of discovered ids and how many times each id has appeared in each possible rank. The number of possible ranks is simply the number of results returned by a single search query, so we begin by applying `collect` a single time in order to determine our database schema. We see that, for our chosen query parameters, there are 109 results.

In [6]:
matches = scrape(url, data)

In [7]:
num_results = len(matches)
num_results

109

Let's start by connecting to our database (the database is created if it doesn't exist yet).

In [8]:
conn = sqlite3.connect('ottiaq.db')
c = conn.cursor()

Next, let's create the table with name `table` that we chose above. We could uncomment and run the following code if we wanted to start from scratch.

In [9]:
# optional: reset table
# query_drop = 'DROP TABLE IF EXISTS {};'.format(table)
# c.execute(query_drop)
# conn.commit()

Our table contains a column for the user ids and one for each possible rank (initialized to 0) for a total of `num_results + 1` matches.

In [10]:
query_create = 'CREATE TABLE IF NOT EXISTS {} (id INTEGER PRIMARY KEY NOT NULL'.format(table)

for i in range(num_results):
    query_create += ',rank{} INTEGER DEFAULT 0'.format(i)

query_create += ');'

In [11]:
c.execute(query_create)
conn.commit()

Next, we insert the ids we found above into the first column of our table. Above, we specified the id as the primary key, which must be unique, so we specify that our `INSERT` query can be ignored (if the id already exists).

In [12]:
query_insert = 'INSERT OR IGNORE INTO {} (id) VALUES (?);'.format(table)
for i in range(num_results):
    c.execute(query_insert, (matches[i],))
conn.commit()

The rank columns' values are all 0 by default, but since we have already queried the search engine 1 time, each row should now have a single rank column set to 1. Later, when we perform repeated searches we will want to update the number of times each id came in each rank, so once again we package the desired sequence of operations into a function before performing it.

In [13]:
def update_ranks(matches):
    for rank, match in enumerate(matches):
        c.execute('SELECT rank{} FROM {} WHERE id=?'.format(rank, table), (match,))
        
        n = c.fetchone()[0] # fetchone returns a tuple
        c.execute('UPDATE {} SET rank{}=? WHERE id=?'.format(table, rank), (n + 1, match))
    conn.commit()

In [14]:
update_ranks(matches)

## Data collection

At this point, most of the work needed to scrape the data has been done. We simply need to iterate the `scrape` and `update_ranks` functions (later, we will discuss the number of times we should do this in order to ensure statistically significant results). As a courtesy, we space out our HTTP requests by a certain wait time.

In [15]:
import time

In [16]:
def collect(num_requests, wait_time=0.5):
    for N in range(num_requests):
        time.sleep(wait_time)

        matches = scrape(url, data)
        update_ranks(matches)
        
        if (N % 50 == 0):
            print("Iteration {}".format(N))

Before running `collect`, we should decide how many requests to make. This will be discussed in the next section based on statistical considerations.

## Statistical analysis

Rather than testing for uniformity of the sampled permutations of members, which would be computationally infeasible, we will test for uniformity of samples found within a given rank, say rank 0.

Consider a general situation in which we are sampling from a categorical distribution with $K$ categories. We wish to test the null hypothesis that category $i$ occurs with probability $p_i$ for $i = 1, \ldots, K$. If this is the case, then for a sample of size $N$, the expected number of observed occurences of category $i$ is $N p_i$. If $N_i$ is the actual number of observations from category $i$, then $(N_i - N p_i)^2$ should be small in order for the null hypothesis to be true. **Pearson's $\chi^2$ test** rejects the null hypothesis if the test statistic
$$\hat\chi^2 = \sum_{i=1}^K \frac{(N_i - N p_i)^2}{N p_i}$$
is sufficiently small.

### Mathematical aside

What constitutes *sufficiently small*? This can be answered by a theorem of Karl Pearson, which states that $\hat\chi^2$ has an approximate $\chi^2$ distribution with $K - 1$ degrees of freedom. A very heuristic explanation involves writing $N_i = \sum_{j=1}^N \mathbf{1}(X_j = i)$ and applying the central limit theorem to see that (assuming the null hypothesis) $\frac{N_i - N p_i}{\sqrt{N p_i}}$ is approximately normally distributed with variance $(1 - p_i)$. Thus, $\hat\chi^2$ is a sum of squares of $K$ *dependent* normal random variables with variance $1 - p_i$. However, an orthogonalization procedure rewrites this as a sum of squares of $K - 1$ *independent* random variables of unit variance, which is a $\chi^2$ random variable with $K - 1$ degrees of freedom.

The upshot is we can use the statistics of $\chi^2$ squared distribution to determine the $p$-value of the $\chi^2$ test according to $p = \mathbf{P}(\chi^2_{K-1} > \hat\chi^2)$, where $\chi^2_{K-1}$ is a $\chi^2$ random variable with $K - 1$ degrees of freedom.

### Testing for uniformity

In order for the results of the $\chi^2$ test to be meaningful, the number of samples $N$ must be sufficiently large. A common heuristic is that samples should be collected until
1. $N p_i \ge 1$ for each $i$ and
2. $N p_i \ge 5$ for at least $80\%$ of the $i$.

In our use case we are testing for the uniform distribution, so the second condition is stronger and we simply require that $N \ge 5 \times \mathrm{num\_results}$.

In case we have already scraped data and stored into our database in the past but forgotten how many samples we took, this number can be recovered by summing along rank columns (actually, the result should be independent of which particular rank column).

In [17]:
def get_ranks(rank):
    return list(map(lambda x: x[0], c.execute('SELECT rank{} FROM fren'.format(rank)).fetchall()))

def num_samples(rank):
    ranks = get_ranks(rank)
    return sum(ranks)

Let's gather twice as much as the minimum required by the heuristic above.

In [18]:
rank = 0
min_samples = 5 * num_results

num_requests = max(0, 2 * min_samples - num_samples(rank))

In [19]:
collect(num_requests)

The hypothesis test can be performed using `scipy.stats`. With the data we've gathered, we can test the null hypothesis for each rank.

In [20]:
import scipy.stats as stats

In [21]:
for rank in range(num_results):
    ranks = get_ranks(rank)
    _, p = stats.chisquare(ranks)
    if p < 0.05:
        print("Reject null hypothesis for rank {} (p = {})".format(rank, p))

Reject null hypothesis for rank 0 (p = 0.030137664878430927)
Reject null hypothesis for rank 26 (p = 0.025711810848352584)
Reject null hypothesis for rank 35 (p = 0.04756631819988459)
Reject null hypothesis for rank 62 (p = 0.048743847357287144)
Reject null hypothesis for rank 70 (p = 0.0020027733042254875)
Reject null hypothesis for rank 88 (p = 0.00279801527514907)


Thus, the null hypothesis is accepted in all but a small handful of cases.